## GRU model for soil moisture prediction (11 years data set)

This script shows the creation of a GRU model for soil moisture prediciton based on air temperature and precipitation. More details can be found in the publication "Gated Recurrent Units for Modelling Time Series of Soil Temperature and Moisture: an Assess-
ment of Performance and Process Reflectivity" from Baumberger et al. (2024).

In [ ]:
import pandas as pd
from pandas import array
import numpy as np
import time
import math
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.dates
from numpy.lib import stride_tricks
from keras.models import Model
from keras.layers import Input, LSTM, Dense, LSTMCell, RNN, Bidirectional, Concatenate, GRU, RepeatVector, TimeDistributed, Dropout, Concatenate, BatchNormalization
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras import callbacks, layers
import tensorflow as tf
print(np.version.version)
print(tf.__version__)

## Data

### Load data set and define variables

In [48]:
windowSize=240

data_row = pd.read_csv("data_st_sm_11_years.csv")

datetime = np.squeeze(data_row["datetime"].to_numpy())
dates = matplotlib.dates.date2num(datetime)

In [49]:
#print(data_row.keys())

at = np.squeeze(data_row["AirTempHourely"].to_numpy())
pr = np.squeeze(data_row["PrecHourely"].to_numpy())
st05 = np.squeeze(data_row["SoilTempHourely"].to_numpy()) 
sm05 = np.squeeze(data_row["SoilMoiHourely"].to_numpy()) 


### Data normalization: single sequences and a whole data set with normalizes data is created

In [50]:
def create_norm_seq(seq):
    seq_norm= (seq-np.min(seq))/(np.max(seq)-np.min(seq))
    return seq_norm

at_norm = create_norm_seq(at)
pr_norm = create_norm_seq(pr)
st05_norm = create_norm_seq(st05)
sm05_norm = create_norm_seq(sm05)

In [51]:
numFeatin = 2
numFeatout = 2

data_norm = np.zeros((len(at),numFeatout+numFeatin))
data_norm[:,0]=at_norm
data_norm[:,1]=pr_norm
data_norm[:,2]=st05_norm
data_norm[:,3]=sm05_norm

### Split data in taining set (7 years) and test set (4 years)

In [52]:
yearly_hours= 365*24
years=7

split = int(windowSize*(np.floor((yearly_hours*years)/windowSize)))
end = int(windowSize*(np.floor((len(data_norm))/windowSize)))

data_train = data_norm[0:split]
data_test = data_norm[split:end]

## Cross validation

### Split training data set for a 4-fold crossvalidation

In [53]:
fold_len = len(data_train)/4

fold_len_div = int(windowSize*(np.floor(fold_len/windowSize)))

In [54]:
data_fold_1 = data_norm[0:fold_len_div]
data_fold_2 = data_norm[fold_len_div:fold_len_div*2]
data_fold_3 = data_norm[fold_len_div*2:fold_len_div*3]
data_fold_4 = data_norm[fold_len_div*3:fold_len_div*4]

### Plot the different folds

In [ ]:
plt.figure(figsize=(15,5))
plt.plot_date(dates,data_norm[:,0],"-",color="lightgrey")
plt.plot_date(dates,data_norm[:,2],"-",color="lightblue")
plt.plot_date(dates[0:fold_len_div],data_fold_1[:,2],"-",color="blue")
plt.plot_date(dates[fold_len_div:fold_len_div*2],data_fold_2[:,2],"-",color="teal")
plt.plot_date(dates[fold_len_div*2:fold_len_div*3],data_fold_3[:,2],"-",color="blue")
plt.plot_date(dates[fold_len_div*3:fold_len_div*4],data_fold_4[:,2],"-",color="seagreen")
plt.plot_date(dates[split:end],data_test[:,2],"-",color="purple")
plt.show()

### Plot example of normalised data

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(data_fold_1[:,0],color="lightgrey")
plt.plot(data_fold_1[:,1],color="lightblue")
plt.plot(data_fold_1[:,2],color="red")
plt.plot(data_fold_1[:,3],color="blue")
plt.show()

### Functions for data preparation of training set and validation set
The data is augmented by a slidiung window approach and the sequences are reshaped for the stateful model architecture, both for the training and validation set.

In [57]:
def create_train_fold(fold1,fold2,fold3,fold_len_div):
    at_train_stack = np.stack((fold1[:,0],fold2[:,0],fold3[:,0]),axis=0)
    pr_train_stack = np.stack((fold1[:,1],fold2[:,1],fold3[:,1]),axis=0)
    st05_train_stack = np.stack((fold1[:,2],fold2[:,2],fold3[:,2]),axis=0)
    sm05_train_stack = np.stack((fold1[:,3],fold2[:,3],fold3[:,3]),axis=0)

    at_train_stack = at_train_stack[:,0:fold_len_div-1]
    pr_train_stack = pr_train_stack[:,0:fold_len_div-1]
    st05_train_stack = st05_train_stack[:,0:fold_len_div-1]
    sm05_train_stack = sm05_train_stack[:,0:fold_len_div-1]

    at_train = stride_tricks.sliding_window_view(at_train_stack,(3,windowSize))
    pr_train = stride_tricks.sliding_window_view(pr_train_stack,(3,windowSize))
    st05_train = stride_tricks.sliding_window_view(st05_train_stack,(3,windowSize))
    sm05_train = stride_tricks.sliding_window_view(sm05_train_stack,(3,windowSize))

    at_train=(at_train.reshape(at_train.shape[1]*3,at_train.shape[0]*windowSize))
    pr_train=(pr_train.reshape(pr_train.shape[1]*3,pr_train.shape[0]*windowSize))
    st05_train=(st05_train.reshape(st05_train.shape[1]*3,st05_train.shape[0]*windowSize))
    sm05_train=(sm05_train.reshape(sm05_train.shape[1]*3,sm05_train.shape[0]*windowSize))

    at_train=(at_train.reshape(at_train.shape[0],at_train.shape[1],1))
    pr_train=(pr_train.reshape(pr_train.shape[0],pr_train.shape[1],1))
    st05_train=(st05_train.reshape(st05_train.shape[0],st05_train.shape[1],1))
    sm05_train=(sm05_train.reshape(sm05_train.shape[0],sm05_train.shape[1],1))

    at_train = 1.*at_train
    pr_train = 1.*pr_train
    st05_train = 1.*st05_train
    sm05_train = 1.*sm05_train
        
    return at_train, pr_train, st05_train, sm05_train

In [58]:

def create_validation_fold(fold):
    val_fold_subset_len = int(windowSize*(np.floor((len(fold)/3)/windowSize)))

    at_val_stack = np.stack((fold[0:val_fold_subset_len,0],fold[val_fold_subset_len:val_fold_subset_len*2,0],fold[val_fold_subset_len*2:val_fold_subset_len*3,0]),axis=0)
    pr_val_stack = np.stack((fold[0:val_fold_subset_len,1],fold[val_fold_subset_len:val_fold_subset_len*2,1],fold[val_fold_subset_len*2:val_fold_subset_len*3,1]),axis=0)
    st05_val_stack = np.stack((fold[0:val_fold_subset_len,2],fold[val_fold_subset_len:val_fold_subset_len*2,2],fold[val_fold_subset_len*2:val_fold_subset_len*3,2]),axis=0)
    sm05_val_stack = np.stack((fold[0:val_fold_subset_len,3],fold[val_fold_subset_len:val_fold_subset_len*2,3],fold[val_fold_subset_len*2:val_fold_subset_len*3,3]),axis=0)

    at_val_stack = at_val_stack[:,0:val_fold_subset_len-1]
    pr_val_stack = pr_val_stack[:,0:val_fold_subset_len-1]
    st05_val_stack = st05_val_stack[:,0:val_fold_subset_len-1]
    sm05_val_stack = sm05_val_stack[:,0:val_fold_subset_len-1]

    at_val = stride_tricks.sliding_window_view(at_val_stack,(3,windowSize))
    pr_val = stride_tricks.sliding_window_view(pr_val_stack,(3,windowSize))
    st05_val = stride_tricks.sliding_window_view(st05_val_stack,(3,windowSize))
    sm05_val = stride_tricks.sliding_window_view(sm05_val_stack,(3,windowSize))

    at_val=(at_val.reshape(at_val.shape[1]*3,at_val.shape[0]*windowSize))
    pr_val=(pr_val.reshape(pr_val.shape[1]*3,pr_val.shape[0]*windowSize))
    st05_val=(st05_val.reshape(st05_val.shape[1]*3,st05_val.shape[0]*windowSize))
    sm05_val=(sm05_val.reshape(sm05_val.shape[1]*3,sm05_val.shape[0]*windowSize))

    at_val=(at_val.reshape(at_val.shape[0],at_val.shape[1],1))
    pr_val=(pr_val.reshape(pr_val.shape[0],pr_val.shape[1],1))
    st05_val=(st05_val.reshape(st05_val.shape[0],st05_val.shape[1],1))
    sm05_val=(sm05_val.reshape(sm05_val.shape[0],sm05_val.shape[1],1))

    at_val = 1.*at_val
    pr_val = 1.*pr_val
    st05_val = 1.*st05_val
    sm05_val = 1.*sm05_val  
    
    return at_val, pr_val, st05_val, sm05_val

### Creating all training and validation sets for the cross validation

In [59]:

at_train_1, pr_train_1, st05_train_1, sm05_train_1 = create_train_fold(fold1=data_fold_2,fold2=data_fold_3,fold3=data_fold_4,fold_len_div=fold_len_div)
at_val_1, pr_val_1, st05_val_1, sm05_val_1 = create_validation_fold(fold=data_fold_1)

at_train_2, pr_train_2, st05_train_2, sm05_train_2 = create_train_fold(fold1=data_fold_3,fold2=data_fold_4,fold3=data_fold_1,fold_len_div=fold_len_div)
at_val_2, pr_val_2, st05_val_2, sm05_val_2 = create_validation_fold(fold=data_fold_2)

at_train_3, pr_train_3, st05_train_3, sm05_train_3 = create_train_fold(fold1=data_fold_4,fold2=data_fold_1,fold3=data_fold_2,fold_len_div=fold_len_div)
at_val_3, pr_val_3, st05_val_3, sm05_val_3 = create_validation_fold(fold=data_fold_3)

at_train_4, pr_train_4, st05_train_4, sm05_train_4 = create_train_fold(fold1=data_fold_1,fold2=data_fold_2,fold3=data_fold_3,fold_len_div=fold_len_div)
at_val_4, pr_val_4, st05_val_4, sm05_val_4 = create_validation_fold(fold=data_fold_4)



## GRU model for cross-validation

### Model creation

In [61]:
batchSize= windowSize*3#240
batchSizeAll= windowSize*4#240
kernel_len=48#int(windowSize/5)
filter_num=32
dropout_rate=0.1



def build_model(batchSize,shape_example):

    initializer = tf.keras.initializers.Constant(1/kernel_len)

    input1=tf.keras.layers.Input(batch_shape=(batchSize,shape_example.shape[1],shape_example.shape[2]))#at_train_1
    input2=tf.keras.layers.Input(batch_shape=(batchSize,shape_example.shape[1],shape_example.shape[2]))#pr_train_1

    conv1d_1 = tf.keras.layers.Conv1D(filters=filter_num,kernel_size=kernel_len,strides=1,activation=None,padding='causal',kernel_initializer=initializer)#
    conv1d_1_output = conv1d_1(input2)

    Concat = tf.keras.layers.Concatenate(axis=-1)([input1,conv1d_1_output])

    GRU1=tf.keras.layers.GRU(16, return_sequences=True,stateful=True,activation='tanh',dropout=dropout_rate)#kernel_regularizer=l2(0.05),recurrent_regularizer=l2(0.05)
    GRU1_output=GRU1(Concat)

    GRU2=tf.keras.layers.GRU(16, return_sequences=True,stateful=True,activation='tanh',dropout=dropout_rate)#kernel_regularizer=l2(0.05),recurrent_regularizer=l2(0.05)
    GRU2_output=GRU2(GRU1_output)

    output1=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(shape_example.shape[2],activation='relu'))(GRU2_output)#sm05_train_1

    model=tf.keras.models.Model(inputs=[input1,input2],outputs=output1)

    conv_weights = conv1d_1.get_weights()
    
    return model, conv_weights


In [62]:
model_1, conv_weights_1 = build_model(batchSize=batchSize,shape_example=at_train_1)
model_2, conv_weights_2 = build_model(batchSize=batchSize,shape_example=at_train_2)
model_3, conv_weights_3 = build_model(batchSize=batchSize,shape_example=at_train_3)
model_4, conv_weights_4 = build_model(batchSize=batchSize,shape_example=at_train_4)



### Print model architecture

In [ ]:
print(model_1.summary())

plot_model(model_1,to_file='model.png',show_shapes=True)

### Model fit for each crossvalidation fold

In [ ]:
epochs= 30
lr_start=0.0025
lr_end=0.0005
factor=(lr_start-lr_end)/epochs
print(factor)
print(lr_start-factor*epochs)

In [65]:
history_dict = dict()

def model_complie_fit(at_train,pr_train,sm05_train,at_val,pr_val,sm05_val,model):
    for i in range(epochs):
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_start-factor*i),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae',tf.keras.metrics.RootMeanSquaredError()]) 
        model.reset_states()
        print(i)
        history_dict['epoch_%i' % i]=model.fit([at_train,pr_train], sm05_train ,epochs=1, batch_size=batchSize, shuffle=False,validation_data=([at_val,pr_val],sm05_val))


### Track training and validation loss

In [66]:

def get_loss():
    history_dict['epoch_0'].history['loss']

    result_loss = []
    result_val_loss = []
    for i in range(epochs):
        result_loss += [history_dict['epoch_%i' % i].history['loss']]
        result_val_loss += [history_dict['epoch_%i' % i].history['val_loss']]

    return result_loss, result_val_loss




### Model training cross-validation 1

In [ ]:

model_complie_fit(at_train_1,pr_train_1,sm05_train_1,at_val_1,pr_val_1,sm05_val_1,model_1)
result_loss_1, result_val_loss_1 = get_loss()
model_1.reset_states()
tf.keras.backend.clear_session()

### Model training cross-validation 2

In [ ]:

model_complie_fit(at_train_2,pr_train_2,sm05_train_2,at_val_2,pr_val_2,sm05_val_2,model_2)
result_loss_2, result_val_loss_2 = get_loss()
model_2.reset_states()
tf.keras.backend.clear_session()

### Model training cross-validation 3

In [ ]:
model_complie_fit(at_train_3,pr_train_3,sm05_train_3,at_val_3,pr_val_3,sm05_val_3,model_3)
result_loss_3, result_val_loss_3 = get_loss()
model_3.reset_states()
tf.keras.backend.clear_session()

### Model training cross-validation 4

In [ ]:
model_complie_fit(at_train_4,pr_train_4,sm05_train_4,at_val_4,pr_val_4,sm05_val_4,model_4)
result_loss_4, result_val_loss_4 = get_loss()
model_4.reset_states()
tf.keras.backend.clear_session()


### Visualisation of training and validation loss for all cross-validation folds

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(result_loss_1,color="teal")
plt.plot(result_val_loss_1,color="navy")
plt.plot(result_loss_2,color="mediumseagreen")
plt.plot(result_val_loss_2,color="indigo")
plt.plot(result_loss_3,color="aqua")
plt.plot(result_val_loss_3,color="mediumpurple")
plt.plot(result_loss_4,color="aquamarine")
plt.plot(result_val_loss_4,color="slateblue")
plt.ylabel('loss', fontsize = 15)
plt.xlabel('epoch', fontsize = 15)
plt.legend(['train_1', 'val_1','train_2', 'val_2','train_3', 'val_3','train_4', 'val_4'])
plt.show()

plt.figure(figsize=(10,5))
plt.plot(result_loss_1,color="red")
plt.plot(result_val_loss_1,color="navy")
plt.ylabel('loss', fontsize = 15)
plt.xlabel('epoch', fontsize = 15)
plt.legend(['train', 'val'])
plt.show()

plt.figure(figsize=(10,5))
plt.plot(result_loss_2,color="red")
plt.plot(result_val_loss_2,color="navy")
plt.ylabel('loss', fontsize = 15)
plt.xlabel('epoch', fontsize = 15)
plt.legend(['train', 'val'])
plt.show()

plt.figure(figsize=(10,5))
plt.plot(result_loss_3,color="red")
plt.plot(result_val_loss_3,color="navy")
plt.ylabel('loss', fontsize = 15)
plt.xlabel('epoch', fontsize = 15)
plt.legend(['train', 'val'])
plt.show()

plt.figure(figsize=(10,5))
plt.plot(result_loss_4,color="red")
plt.plot(result_val_loss_4,color="navy")
plt.ylabel('loss', fontsize = 15)
plt.xlabel('epoch', fontsize = 15)
plt.legend(['train', 'val'])
plt.show()

## GRU model final (the final model is based on all folds)

### Functions for data preparation of training set based on all folds
The data is augmented by a slidiung window approach and the sequences are reshaped for the stateful model architecture for the training set based on all folds.

In [ ]:
def create_train_fold_all(fold1,fold2,fold3,fold4,fold_len_div):
    at_train_stack = np.stack((fold1[:,0],fold2[:,0],fold3[:,0],fold4[:,0]),axis=0)
    pr_train_stack = np.stack((fold1[:,1],fold2[:,1],fold3[:,1],fold4[:,1]),axis=0)
    st05_train_stack = np.stack((fold1[:,2],fold2[:,2],fold3[:,2],fold4[:,2]),axis=0)
    sm05_train_stack = np.stack((fold1[:,3],fold2[:,3],fold3[:,3],fold4[:,3]),axis=0)

    at_train_stack = at_train_stack[:,0:fold_len_div-1]
    pr_train_stack = pr_train_stack[:,0:fold_len_div-1]
    st05_train_stack = st05_train_stack[:,0:fold_len_div-1]
    sm05_train_stack = sm05_train_stack[:,0:fold_len_div-1]

    at_train = stride_tricks.sliding_window_view(at_train_stack,(4,windowSize))
    pr_train = stride_tricks.sliding_window_view(pr_train_stack,(4,windowSize))
    st05_train = stride_tricks.sliding_window_view(st05_train_stack,(4,windowSize))
    sm05_train = stride_tricks.sliding_window_view(sm05_train_stack,(4,windowSize))

    at_train=(at_train.reshape(at_train.shape[1]*4,at_train.shape[0]*windowSize))
    pr_train=(pr_train.reshape(pr_train.shape[1]*4,pr_train.shape[0]*windowSize))
    st05_train=(st05_train.reshape(st05_train.shape[1]*4,st05_train.shape[0]*windowSize))
    sm05_train=(sm05_train.reshape(sm05_train.shape[1]*4,sm05_train.shape[0]*windowSize))

    at_train=(at_train.reshape(at_train.shape[0],at_train.shape[1],1))
    pr_train=(pr_train.reshape(pr_train.shape[0],pr_train.shape[1],1))
    st05_train=(st05_train.reshape(st05_train.shape[0],st05_train.shape[1],1))
    sm05_train=(sm05_train.reshape(sm05_train.shape[0],sm05_train.shape[1],1))

    at_train = 1.*at_train
    pr_train = 1.*pr_train
    st05_train = 1.*st05_train
    sm05_train = 1.*sm05_train
        
    return at_train, pr_train, st05_train, sm05_train

In [ ]:
at_train_all, pr_train_all, st05_train_all, sm05_train_all = create_train_fold_all(fold1=data_fold_1,fold2=data_fold_2,fold3=data_fold_3,fold4=data_fold_4,fold_len_div=fold_len_div)

### Create model for the training data set based on all folds
The data shape is different since four insteard of three sequences were stacked

In [ ]:
model_all, conv_weights_all = build_model(batchSize=batchSizeAll,shape_example=at_train_all)

### Model fit

In [ ]:

history_dict = dict()

def model_complie_fit_all(at_train,pr_train,sm05_train_all,model):
    for i in range(epochs):
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_start-factor*i),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae',tf.keras.metrics.RootMeanSquaredError()]) 
        model.reset_states()
        print(i)
        history_dict['epoch_%i' % i]=model.fit([at_train,pr_train], sm05_train_all ,epochs=1, batch_size=batchSizeAll, shuffle=False)
    weights =  model.get_weights()
    
    return model, weights

### Function for tracking training loss

In [ ]:
def get_loss_all():
    history_dict['epoch_0'].history['loss']

    result_loss = []
    for i in range(epochs):
        result_loss += [history_dict['epoch_%i' % i].history['loss']]
        
    return result_loss

### Model training

In [ ]:
model_all, model_weights = model_complie_fit_all(at_train_all,pr_train_all,sm05_train_all,model_all)

### Visualisation of the training loss and convolutional kernel

In [ ]:
result_loss_all= get_loss_all()

plt.figure(figsize=(10,5))
plt.plot(result_loss_all)
plt.ylabel('loss', fontsize = 15)
plt.xlabel('epoch', fontsize = 15)
plt.legend(['train'])
plt.show()

plt.figure(figsize=(10,5))
plt.plot(model_weights[0].reshape(kernel_len*filter_num)[0::filter_num],color="#08306b")
plt.ylabel('weight', fontsize = 15)
plt.xlabel('sequence position', fontsize = 15)
plt.show()

model_all.reset_states()

## Model prediction on the test set (4 years)

For the prediction of sequences the same model has to be build, but with a batch size of 1. 

In [ ]:
batchSizeNew=1

def build_model_prediction(model):

    initializer = tf.keras.initializers.Constant(1/kernel_len)

    input1=tf.keras.layers.Input(batch_shape=(batchSizeNew,at_train_1.shape[1],at_train_1.shape[2]))
    input2=tf.keras.layers.Input(batch_shape=(batchSizeNew,pr_train_1.shape[1],pr_train_1.shape[2]))

    conv1d_1 = tf.keras.layers.Conv1D(filters=filter_num,kernel_size=kernel_len,strides=1,activation=None,padding='causal',kernel_initializer=initializer)#
    conv1d_1_output = conv1d_1(input2)

    Concat = tf.keras.layers.Concatenate(axis=-1)([input1,conv1d_1_output])#conv1d_1_output

    GRU1=tf.keras.layers.GRU(16, return_sequences=True,stateful=True,activation='tanh',dropout=dropout_rate)#kernel_regularizer=l2(0.05),recurrent_regularizer=l2(0.05)
    GRU1_output=GRU1(Concat)

    GRU2=tf.keras.layers.GRU(16, return_sequences=True,stateful=True,activation='tanh',dropout=dropout_rate)#kernel_regularizer=l2(0.05),recurrent_regularizer=l2(0.05)
    GRU2_output=GRU2(GRU1_output)

    output1=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(sm05_train_1.shape[2],activation='relu'))(GRU2_output)

    new_model=tf.keras.models.Model(inputs=[input1,input2],outputs=output1)#,output2,output3

    old_weights = model.get_weights()
    new_model.set_weights(old_weights)

    new_model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.MeanSquaredError()) 

    return new_model

In [ ]:
new_model_1 = build_model_prediction(model_1)
new_model_2 = build_model_prediction(model_2)
new_model_3 = build_model_prediction(model_3)
new_model_4 = build_model_prediction(model_4)

new_model_all = build_model_prediction(model_all)

### Reshaping of data for prediction

In [ ]:

def predict_sequence(input_array_1,input_array_2,new_model):
    seqNum=np.floor(len(input_array_1)/windowSize)

    data2a = input_array_1[:int(seqNum)*windowSize]
    x1=np.split(data2a,seqNum)
    x1=np.stack(x1,axis=1)

    data2b = input_array_2[:int(seqNum)*windowSize]
    x2=np.split(data2b,seqNum)
    x2=np.stack(x2,axis=1)

    x1 = np.transpose(x1)
    x1 = x1.reshape(x1.shape[0],x1.shape[1],1)
    x2 = np.transpose(x2)
    x2 = x2.reshape(x2.shape[0],x2.shape[1],1)
    
    year_predict = new_model.predict([x1,x2],batch_size=1)

    year_predict = np.squeeze(year_predict)

    year_predict = year_predict.ravel()
    return year_predict


## Error estimation (validation error) and visualisation

### Cross-validation

In [ ]:
def calculate_val_error(validation_fold,position,new_model):
    
    #error calculatioin

    data_val_at = validation_fold[:,0]
    data_val_pr = validation_fold[:,1]

    val_pred = predict_sequence(data_val_at,data_val_pr,new_model)

    true_norm=sm05_norm[fold_len_div*(position-1):fold_len_div*position]
    pred_norm=val_pred

    true=sm05[fold_len_div*(position-1):fold_len_div*position]
    pred=((val_pred*(np.max(sm05)-np.min(sm05)))+np.min(sm05))
    
    MAE = np.mean(np.abs(true-pred))
    print("MAE", MAE)
    
    MAE_norm = np.mean(np.abs(true_norm-pred_norm))
    print("MAE norm", MAE_norm)
 
    def rmse(pred, true):
        return np.sqrt(((pred - true)**2).mean())

    RSME = rmse(pred, true)
    print("RSME", RSME)

    RSME_norm = rmse(pred_norm,true_norm)
    print("RSME_norm", RSME_norm)

    corr_matrix = np.corrcoef(true, pred)
    corr = corr_matrix[0,1]
    R_sq = corr**2
    print("R²", R_sq)

    #plot

    datetime = np.squeeze(data_row["datetime"].to_numpy())
    dates_test = matplotlib.dates.date2num(datetime[fold_len_div*(position-1):fold_len_div*position])

    fig, ax = plt.subplots(figsize=(20,5))
    fig.subplots_adjust(right=0.75)

    twin1 = ax.twinx()
    twin2 = ax.twinx()
    twin3 = ax.twinx()

    twin2.spines.right.set_position(("axes", 1.07))

    p2, = twin2.plot_date(dates_test,(data_val_pr*(np.max(pr)-np.min(pr)))+np.min(pr),"-",color="deepskyblue", alpha= 0.25 ,label="Input: Precipitation")
    p1, = twin1.plot_date(dates_test,(data_val_at*(np.max(at)-np.min(at)))+np.min(at),"-",color="grey", alpha= 0.25,label="Input: Air temperature")
    p3, = ax.plot_date(dates_test,(val_pred*(np.max(sm05)-np.min(sm05)))+np.min(sm05),"-",color="royalblue",label="Prediction: Soil moisture")
    p4, = ax.plot_date(dates_test,sm05[fold_len_div*(position-1):fold_len_div*position],"-",color="darkblue",label="Truth: Soil moisture")#[(total_len-end):(total_len)]
    p5, = twin3.plot_date(dates_test,sm05[fold_len_div*(position-1):fold_len_div*position],"-",color="darkblue",label="Truth")#[(total_len-end):(total_len)]
    p5, = twin3.plot_date(dates_test,(val_pred*(np.max(sm05)-np.min(sm05)))+np.min(sm05),"-",color="royalblue",label="Prediction")

    ax.set_xlabel("Date")
    ax.set_ylabel("Soil moisture [mm]")
    twin1.set_ylabel("Air temperature [°C]")
    twin2.set_ylabel("Precipitation [mm]")

    twin3.axes.get_yaxis().set_visible(False)

    ax.xaxis.label.set_size(12)
    ax.yaxis.label.set_size(12)
    twin1.yaxis.label.set_size(12)
    twin2.yaxis.label.set_size(12)

    ax.yaxis.label.set_color("royalblue")#p1.get_color()
    twin1.yaxis.label.set_color("grey")
    twin2.yaxis.label.set_color("deepskyblue")

    tkw = dict(size=4, width=2, labelsize=12)
    ax.tick_params(axis='y', colors="royalblue", **tkw)
    twin1.tick_params(axis='y', colors="grey", **tkw)
    twin2.tick_params(axis='y', colors="deepskyblue", **tkw)
    ax.tick_params(axis='x', **tkw)

    leg = ax.legend(handles=[p3, p4, p1, p2],prop={'size': 12})
    leg.get_lines()[0].set_linewidth(4)
    leg.get_lines()[1].set_linewidth(4)
    leg.get_lines()[2].set_linewidth(4)
    leg.get_lines()[3].set_linewidth(4)
    leg.remove()
    twin3.add_artist(leg)

    plt.show()

    return MAE, RSME, R_sq

In [ ]:
MAE_1, RSME_1, R_sq_1 = calculate_val_error(validation_fold=data_fold_1,position=1,new_model=new_model_1)

MAE_2, RSME_2, R_sq_2 = calculate_val_error(validation_fold=data_fold_2,position=2,new_model=new_model_2)

MAE_3, RSME_3, R_sq_3 = calculate_val_error(validation_fold=data_fold_3,position=3,new_model=new_model_3)

MAE_4, RSME_4, R_sq_4 = calculate_val_error(validation_fold=data_fold_4,position=4,new_model=new_model_4)

### Error estimation - mean of all cross-validation folds

In [ ]:
print(MAE_1)
print(MAE_2)
print(MAE_3)
print(MAE_4)

mean_MAE = (MAE_1+MAE_2+MAE_3+MAE_4)/4
print("mean_MAE",mean_MAE)

mean_RSME = (RSME_1+RSME_2+RSME_3+RSME_4)/4
print("mean_RSME",mean_RSME)

mean_R_sq = (R_sq_1+R_sq_2+R_sq_3+R_sq_4)/4
print("mean_R_sq",mean_R_sq)

range_sm05 = max(sm05)-min(sm05)

MAE_val_percent = 100/range_sm05*mean_MAE
print("MAE_val_percent", MAE_val_percent)

## Error estimation (test error) and visualisation 

(visualisation include 4 years sequence, half year sequence and hexbin plot)

In [ ]:
def calculate_test_error(test_set,new_model):
    
    #error calculatioin

    data_test_at = test_set[:,0]
    data_test_pr = test_set[:,1]

    test_pred = predict_sequence(data_test_at,data_test_pr,new_model)

    true_norm=sm05_norm[split:end]
    pred_norm=test_pred

    true=sm05[split:end]
    pred=((test_pred*(np.max(sm05)-np.min(sm05)))+np.min(sm05))

    MAE = np.mean(np.abs(true-pred))
    print("MAE", MAE)
    
    MAE_norm = np.mean(np.abs(true_norm-pred_norm))
    print("MAE norm", MAE_norm)
 
    def rmse(pred, true):
        return np.sqrt(((pred - true)**2).mean())

    RSME = rmse(pred, true)
    print("RSME", RSME)

    RSME_norm = rmse(pred_norm,true_norm)
    print("RSME_norm", RSME_norm)

    corr_matrix = np.corrcoef(true, pred)
    corr = corr_matrix[0,1]
    R_sq = corr**2
    print("R²", R_sq)

    #plot

    datetime = np.squeeze(data_row["datetime"].to_numpy())
    dates_test = matplotlib.dates.date2num(datetime[split:end])

    fig, ax = plt.subplots(figsize=(20,5))
    fig.subplots_adjust(right=0.75)

    twin1 = ax.twinx()
    twin2 = ax.twinx()
    twin3 = ax.twinx()

    twin2.spines.right.set_position(("axes", 1.07))

    p2, = twin2.plot_date(dates_test,(data_test_pr*(np.max(pr)-np.min(pr)))+np.min(pr),"-",color="skyblue", alpha= 0.25 ,label="Input: Precipitation")
    p1, = twin1.plot_date(dates_test,(data_test_at*(np.max(at)-np.min(at)))+np.min(at),"-",color="palevioletred", alpha= 0.2,label="Input: Air temperature")
    p3, = ax.plot_date(dates_test,(test_pred*(np.max(sm05)-np.min(sm05)))+np.min(sm05),"-",color="royalblue",label="Prediction: Soil moisture")
    p4, = ax.plot_date(dates_test,sm05[split:end],"-",color="darkblue",label="Truth: Soil moisture")#[(total_len-end):(total_len)]
    p5, = twin3.plot_date(dates_test,sm05[split:end],"-",color="darkblue",label="Truth")#[(total_len-end):(total_len)]
    p5, = twin3.plot_date(dates_test,(test_pred*(np.max(sm05)-np.min(sm05)))+np.min(sm05),"-",color="royalblue",label="Prediction")

    ax.margins(x=0)

    ax.set_xlabel("Date")
    ax.set_ylabel("Soil moisture [mm]")
    twin1.set_ylabel("Air temperature [°C]")
    twin2.set_ylabel("Precipitation [mm]")

    twin3.axes.get_yaxis().set_visible(False)

    ax.xaxis.label.set_size(15)
    ax.yaxis.label.set_size(15)
    twin1.yaxis.label.set_size(15)
    twin2.yaxis.label.set_size(15)

    ax.yaxis.label.set_color("royalblue")#p1.get_color()
    twin1.yaxis.label.set_color("palevioletred")
    twin2.yaxis.label.set_color("skyblue")

    tkw = dict(size=4, width=2, labelsize=15)
    ax.tick_params(axis='y', colors="royalblue", **tkw)
    twin1.tick_params(axis='y', colors="palevioletred", **tkw)
    twin2.tick_params(axis='y', colors="skyblue", **tkw)
    ax.tick_params(axis='x', **tkw)

    leg = ax.legend(handles=[p3, p4, p1, p2],prop={'size': 15},loc='upper right')
    leg.get_lines()[0].set_linewidth(4)
    leg.get_lines()[1].set_linewidth(4)
    leg.get_lines()[2].set_linewidth(4)
    leg.get_lines()[3].set_linewidth(4)
    leg.remove()
    twin3.add_artist(leg)

    plt.show()



    fig, ax = plt.subplots(figsize=(20,5))
    fig.subplots_adjust(right=0.75)

    twin1 = ax.twinx()
    twin2 = ax.twinx()
    twin3 = ax.twinx()

    twin2.spines.right.set_position(("axes", 1.07))

    p2, = twin2.plot_date(dates_test[18350:21550],(data_test_pr[18350:21550]*(np.max(pr)-np.min(pr)))+np.min(pr),"-",color="skyblue", alpha= 0.25 ,label="Input: Precipitation")#18500 21500
    p1, = twin1.plot_date(dates_test[18350:21550],(data_test_at[18350:21550]*(np.max(at)-np.min(at)))+np.min(at),"-",color="palevioletred", alpha= 0.2,label="Input: Air temperature")
    p3, = ax.plot_date(dates_test[18350:21550],(test_pred[18350:21550]*(np.max(sm05)-np.min(sm05)))+np.min(sm05),"-",color="royalblue",label="Prediction: Soil moisture")
    p4, = ax.plot_date(dates_test[18350:21550],sm05[split:end][18350:21550],"-",color="darkblue",label="Truth: Soil moisture")#[(total_len-end):(total_len)]
    p5, = twin3.plot_date(dates_test[18350:21550],sm05[split:end][18350:21550],"-",color="darkblue",label="Truth")#[(total_len-end):(total_len)]
    p5, = twin3.plot_date(dates_test[18350:21550],(test_pred[18350:21550]*(np.max(sm05)-np.min(sm05)))+np.min(sm05),"-",color="royalblue",label="Prediction")

    ax.margins(x=0)

    year_month_formatter = matplotlib.dates.DateFormatter("%Y-%m")
    ax.xaxis.set_major_formatter(year_month_formatter)
    ax.xaxis.set_major_locator(matplotlib.dates.WeekdayLocator(interval=4))

    ax.set_xlabel("Date")
    ax.set_ylabel("Soil moisture [mm]")
    twin1.set_ylabel("Air temperature [°C]")
    twin2.set_ylabel("Precipitation [mm]")

    twin3.axes.get_yaxis().set_visible(False)

    ax.xaxis.label.set_size(15)
    ax.yaxis.label.set_size(15)
    twin1.yaxis.label.set_size(15)
    twin2.yaxis.label.set_size(15)

    ax.yaxis.label.set_color("royalblue")#p1.get_color()
    twin1.yaxis.label.set_color("palevioletred")
    twin2.yaxis.label.set_color("skyblue")

    tkw = dict(size=4, width=2, labelsize=15)
    ax.tick_params(axis='y', colors="royalblue", **tkw)
    twin1.tick_params(axis='y', colors="palevioletred", **tkw)
    twin2.tick_params(axis='y', colors="skyblue", **tkw)
    ax.tick_params(axis='x', **tkw)

    leg = ax.legend(handles=[p3, p4, p1, p2],prop={'size': 15},loc='upper right')
    leg.get_lines()[0].set_linewidth(4)
    leg.get_lines()[1].set_linewidth(4)
    leg.get_lines()[2].set_linewidth(4)
    leg.get_lines()[3].set_linewidth(4)
    leg.remove()
    twin3.add_artist(leg)

    plt.show()




    plt.figure(figsize=(5,5))
    plt.subplots_adjust(left=0.2, right=0.9, top=0.9, bottom=0.2)
    plt.hexbin(true,pred,gridsize = [25,8],  cmap ="bone_r",mincnt=1,bins='log')#bins='log',
    plt.xlabel("Soil moisture truth [mm]",fontsize=15) 
    plt.ylabel("Soil moisture prediction [mm]",fontsize=15)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.plot([10,45], [10, 45], ls="--", c=".3")
    plt.xlim(12,44)
    plt.ylim(12,44)
    plt.show()




    return MAE, RSME, R_sq

In [ ]:
MAE_test, RSME_test, R_sq_test = calculate_test_error(test_set=data_test,new_model=new_model_all)

MAE_test_percent = 100/range_sm05*MAE_test
print("MAE_test_percent", MAE_test_percent)

## Save model

In [ ]:
# new_model_all.save('') #fill in directory